Курсовой проект для курса "Python для Data Science"

Материалы к проекту (файлы):
train.csv
test.csv

Задание:
Используя данные из train.csv, построить
модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели предсказать
цены для квартир из файла test.csv.

Целевая переменная:
Price

Метрика:
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Сдача проекта:
1. Прислать в раздел Задания Урока 10 ("Вебинар. Консультация по итоговому проекту")
ссылку на программу в github (программа должна содержаться в файле Jupyter Notebook
с расширением ipynb). (Pull request не нужен, только ссылка ведущая на сам скрипт).
2. Приложить файл с названием по образцу Khodyreva_predictions.csv
с предсказанными ценами для квартир из test.csv (файл должен содержать два поля: Id, Price).
В файле с предсказаниями должна быть 5001 строка (шапка + 5000 предсказаний).

Сроки и условия сдачи:
Дедлайн: сдать проект нужно в течение 72 часов после начала Урока 10 ("Вебинар. Консультация по итоговому проекту").
Для успешной сдачи должны быть все предсказания (для 5000 квартир) и R2 должен быть больше 0.6.
При сдаче до дедлайна результат проекта может попасть в топ лучших результатов.
Повторная сдача и проверка результатов возможны только при условии предыдущей неуспешной сдачи.
Успешный проект нельзя пересдать в целях повышения результата.
Проекты, сданные после дедлайна или сданные повторно, не попадают в топ лучших результатов, но можно узнать результат.
В качестве итогового результата берется первый успешный результат, последующие успешные результаты не учитываются.

Примечание:
Все файлы csv должны содержать названия полей (header - то есть "шапку"),
разделитель - запятая. В файлах не должны содержаться индексы из датафрейма.

Рекомендации для файла с кодом (ipynb):
1. Файл должен содержать заголовки и комментарии
2. Повторяющиеся операции лучше оформлять в виде функций
3. Не делать вывод большого количества строк таблиц (5-10 достаточно)
4. По возможности добавлять графики, описывающие данные (около 3-5)
5. Добавлять только лучшую модель, то есть не включать в код все варианты решения проекта
6. Скрипт проекта должен отрабатывать от начала и до конца (от загрузки данных до выгрузки предсказаний)
7. Весь проект должен быть в одном скрипте (файл ipynb).
8. При использовании статистик (среднее, медиана и т.д.) в качестве признаков,
лучше считать их на трейне, и потом на валидационных и тестовых данных не считать
статистики заново, а брать их с трейна. Если хватает знаний, можно использовать кросс-валидацию,
но для сдачи этого проекта достаточно разбить данные из train.csv на train и valid.
9. Проект должен полностью отрабатывать за разумное время (не больше 10 минут),
поэтому в финальный вариант лучше не включать GridSearch с перебором
большого количества сочетаний параметров.
10. Допускается применение библиотек Python и моделей машинного обучения,
которые были в курсе Python для Data Science. Градиентный бустинг изучается
в последующих курсах, поэтому в этом проекте его применять не следует.
Самая сложная из допустимых моделей - RandomForestRegressor из sklearn.

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import r2_score
import warnings

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100

%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

In [2]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

data_train.shape, data_test.shape

((10000, 20), (5000, 19))

In [3]:
# Смотрим на типы данных
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

In [4]:
data_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [6]:
# Проверка на наличие пропусков в тренировочных данных
print("\nMissing Values in Train Data:")
print(data_train.isnull().sum())


Missing Values in Train Data:
Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64


In [7]:
# Заполнение пропусков медианой в числовых признаках
data_train['LifeSquare'].fillna(data_train['LifeSquare'].median(), inplace=True)
data_train['Healthcare_1'].fillna(data_train['Healthcare_1'].median(), inplace=True)

data_test['LifeSquare'].fillna(data_test['LifeSquare'].median(), inplace=True)
data_test['Healthcare_1'].fillna(data_test['Healthcare_1'].median(), inplace=True)

In [9]:
# Проверка, что пропуски устранены
print("Missing Values in Train Data:")
print(data_train.isnull().sum())

print("\nMissing Values in Test Data:")
print(data_test.isnull().sum())


Missing Values in Train Data After Imputation:
Id               0
DistrictId       0
Rooms            0
Square           0
LifeSquare       0
KitchenSquare    0
Floor            0
HouseFloor       0
HouseYear        0
Ecology_1        0
Ecology_2        0
Ecology_3        0
Social_1         0
Social_2         0
Social_3         0
Healthcare_1     0
Helthcare_2      0
Shops_1          0
Shops_2          0
Price            0
dtype: int64

Missing Values in Test Data After Imputation:
Id               0
DistrictId       0
Rooms            0
Square           0
LifeSquare       0
KitchenSquare    0
Floor            0
HouseFloor       0
HouseYear        0
Ecology_1        0
Ecology_2        0
Ecology_3        0
Social_1         0
Social_2         0
Social_3         0
Healthcare_1     0
Helthcare_2      0
Shops_1          0
Shops_2          0
dtype: int64


In [10]:
# изменим типы данных для Rooms и для HouseFloor

data_train['Rooms'] = data_train['Rooms'].astype('int64')
data_train['HouseFloor'] = data_train['HouseFloor'].astype('int64')

data_test['Rooms'] = data_test['Rooms'].astype('int64')
data_test['HouseFloor'] = data_test['HouseFloor'].astype('int64')

In [11]:
# количество комнат
data_train['Rooms'].value_counts()

,count
Rooms,
2,3880
1,3705
3,2235
4,150
5,18
0,8
10,2
19,1
6,1


In [15]:
# One-Hot Encoding для категориальных признаков
data_train = pd.get_dummies(data_train, columns=['Ecology_2', 'Ecology_3', 'Shops_2'], drop_first=True)
data_test = pd.get_dummies(data_test, columns=['Ecology_2', 'Ecology_3', 'Shops_2'], drop_first=True)

In [18]:
# Разделение данных на признаки и целевую переменную
X = data_train.drop(columns=['Price', 'Id'])
y = data_train['Price']

# Разделение данных на обучающую и валидационную выборки (80/20)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели RandomForestRegressor
model = RFR(random_state=42)
model.fit(X_train, y_train)

# Предсказание на валидационной выборке
y_pred = model.predict(X_valid)

# Оценка модели
r2 = r2_score(y_valid, y_pred)
print(f'R2 Score: {r2:.4f}')

R2 Score: 0.7121


R2 = 0.7121  хороший показатель.

In [17]:
# Предсказание цен на тестовом наборе данных
test_predictions = model.predict(data_test.drop(columns=['Id']))

# Создание DataFrame с результатами
Pred_data = pd.DataFrame({
    'Id': data_test['Id'],
    'Price': test_predictions
})

# Сохранение результатов в файл
Pred_data.to_csv('/content/Khodyreva_predictions.csv', index=False)

print("Файл Khodyreva_predictions.csv готов")


Файл Khodyreva_predictions.csv готов
